In [ ]:
from copy import copy
from json_numpy import default, object_hook
import json
import json_numpy as jnp
import jsons
import numpy as np


In [ ]:
from pprint import pprint, pformat


def pformat_type(obj) -> str:
    return pformat((type(obj), obj))


def print_type(obj) -> None:
    print(pformat_type(obj))


In [ ]:
# a numpy array
a = np.array([1, 2, 3, 4, 5])
a


In [ ]:
# dumping with jsons a np array does not work
# jsons.dump(a)


In [ ]:
# fails: dump() missing 1 required positional argument: 'fp'
# a_jnp = jnp.dump(a)
# type(a_jnp), a_jnp


In [ ]:
# dumping with json_numpy returns a string
a_jnp = jnp.dumps(a)
print_type(a_jnp)


In [ ]:
# we can reload the string into a numpy array
a_jnp_np = jnp.loads(a_jnp)
print_type(a_jnp_np)


In [ ]:
# we can load the string to turn it into a dict
a_jnp_load = json.loads(a_jnp)
print_type(a_jnp_load)


In [ ]:
# if we want to convert the dict into a numpy
# we have to dump it back to string
print_type(jnp.loads(json.dumps(a_jnp_load)))


In [ ]:
# passing the default json_np func as a serializer does not work
# jsons.set_serializer(default, np.ndarray)

# ??
# jsons.dump(a, default=default)

# ??
# default_wrap = lambda obj, *_, **__: default(obj)
# jsons.dump(a, default=default_wrap)


In [ ]:
# a sample class with attributes
class Sample:
    def __init__(self) -> None:
        self.a = 1
        self.l = [1, 2, 3]

    def __str__(self) -> str:
        # return f"{self.a} {self.l}"
        return f"{self.a} {pformat_type(self.l)}"

    def __repr__(self) -> str:
        return self.__str__()


s = Sample()


In [ ]:
# jsons works perfectly for this
js = jsons.dump(s)  # serialize
print_type(js)
ls = jsons.load(js, Sample)  # deserialize
print_type(ls)


In [ ]:
# a sample class with a numpy attribute
class SampleNp:
    def __init__(self) -> None:
        self.a = 1
        self.l = [1, 2, 3]
        self.npl: np.ndarray = np.array(self.l)

    def __str__(self) -> str:
        return f"{self.a} {self.l} {pformat_type(self.npl)}"

    def __repr__(self) -> str:
        return self.__str__()


snp = SampleNp()


In [ ]:
def np_serializer(
    obj: np.ndarray,
    **kwargs,
) -> str:
    """A serializer for numpy arrays."""
    print(f"serializing {obj}")

    return jnp.dumps(obj)

    # obj_jnp: str = jnp.dumps(obj)
    # return json.loads(obj_jnp)


jsons.set_serializer(np_serializer, np.ndarray)


In [ ]:
# we can dump with jsons using the custom serializer
# which will create a string, but that's ok for serialization
a_j = jsons.dump(a)
print_type(a_j)


In [ ]:
# the custom serializer does work inside the class as is
# no need to define a custom serializer for the class
snp_j = jsons.dump(snp)
print_type(snp_j)


In [ ]:
def np_deserializer(
    obj: str,
    cls: type = np.ndarray,
    **kwargs,
):
    """A deserializer for numpy arrays."""
    print(f"deserializing ({type(obj)}) {obj}")
    return jnp.loads(obj)


jsons.set_deserializer(np_deserializer, np.ndarray)


In [ ]:
# we take a string and use jsons to convert it to numpy using json_np internally
a_j_load = jsons.load(a_j, np.ndarray)
print_type(a_j_load)


In [ ]:
# we can load a class, but the numpy array stays a string
snp_l = jsons.load(snp_j, SampleNp)
print_type(snp_l)

# we can patch it manually
snp_l.npl = jnp.loads(snp_l.npl)
print_type(snp_l)


In [ ]:
# we can turn the string into a dict and patch it in the sample_np dump
# so that while decoding we can do something like if "__numpy__" in obj

snp_j_npl_load = json.loads(snp_j["npl"])
print_type(snp_j_npl_load)

snp_j_copy = copy(snp_j)
snp_j_copy["npl"] = snp_j_npl_load
print_type(snp_j_copy)


In [ ]:
# if we just load it the npl stays a dict
snp_l = jsons.load(snp_j_copy, SampleNp)
print_type(snp_l)


In [ ]:
# if we just load it the npl stays a string
snp_l = jsons.load(snp_j, SampleNp)
type(snp_l), snp_l.npl


In [ ]:
# # a custom deserializer should work but who knows how to implement it
# def sample_np_deserializer(
#     obj: str,
#     cls: type = np.ndarray,
#     **kwargs,
# ):
#     """"""
#     print(f"deserializing ({type(obj)}) {obj}")
#     if isinstance(obj, dict) and "__numpy__" in obj:
#         # return jnp.loads(obj)
#         return jsons.load(obj, np.ndarray)
#     return jsons.default_deserializer(obj, cls, **kwargs) ??
# jsons.set_deserializer(sample_np_deserializer, SampleNp)
# snp_l = jsons.load(snp_j_copy, SampleNp)
# type(snp_l), snp_l


In [ ]:
# # patch the class after loading
# def sample_np_deserializer(
#     obj: str,
#     cls: type = np.ndarray,
#     **kwargs,
# ):
#     """"""
#     print(f"deserializing ({type(obj)}) {obj}")
#     # sample_np_obj = jsons.load(obj, SampleNp) # duh
#     sample_np_obj.npl = jsons.load(sample_np_obj.npl, np.ndarray)
#     return sample_np_obj
# jsons.set_deserializer(sample_np_deserializer, SampleNp)
# snp_l = jsons.load(snp_j, SampleNp)
# print_type(snp_l)


In [ ]:
def load_sample_np(obj) -> SampleNp:
    """Load a SampleNp object from an object, then patch the numpy attributes."""
    snp_l = jsons.load(obj, SampleNp)
    # patch it manually
    snp_l.npl = jnp.loads(snp_l.npl)
    return snp_l

snp_l_patch = load_sample_np(snp_j)
print_type(snp_l_patch)


In [ ]:
# dump the class as a string
snp_js = jsons.dumps(snp)
print_type(snp_js)


In [ ]:
def loads_sample_np(obj_str) -> SampleNp:
    """Load a SampleNp object from a string, then patch the numpy attributes."""
    snp_l = jsons.loads(obj_str, SampleNp)
    # patch it manually
    snp_l.npl = jnp.loads(snp_l.npl)
    return snp_l


loads_sample_np(snp_js)
